In [ ]:
%pip install opencage pandas plotly requests pandas #install libraries


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [55]:
#import libraries here:
import requests
import pandas as pd

In [38]:
url = 'https://api.open-meteo.com/v1/forecast' # api url which im using a get request

def get_forecast_hourly(lat, long, forecast_d=5): #creating an api call for user's input
    params = {
        'latitude': lat,  
        'longitude': long, #have to convert city to lat,long
        'forecast_days': forecast_d, #number of days in forecast
        'temperature_unit': 'fahrenheit', # Change unit maybe later according to user pref
        'precipitation_unit': 'mm',
        'windspeed_unit': 'kmh', 
        'hourly': 'temperature_2m,precipitation',
        'timezone': 'America/New_York',  # Change according to your timezone - later user could maybe do this 
    }
    response = requests.get(url, params=params)
    return response.json()

In [39]:
from opencage.geocoder import OpenCageGeocode
key = '4e0e9443673d4093ab7e797721508ea3'  # my api key
geocoder = OpenCageGeocode(key) #gets lat,long from city

def get_cord(city):
    response =  geocoder.geocode(city)
    if  response:
        if len(response):
            return response[0]['geometry']['lat'], response[0]['geometry']['lng']
    else:
        print('city is not found')
        return None, None
    
# TODO: Add zip code search

In [40]:
city = input("Enter the city: ")  # user input
lat, long = get_cord(city)

if lat and long:
    forecast_data = get_forecast_hourly(lat, long)
    forecast_df = pd.DataFrame(forecast_data['hourly'])

#TODO: Add suggestions for city names

In [41]:
forecast_df #print out 5 day forecast per hour

,time,temperature_2m,precipitation
0,2023-06-08T00:00,57.6,0.0
1,2023-06-08T01:00,56.1,0.0
2,2023-06-08T02:00,55.3,0.0
3,2023-06-08T03:00,53.7,0.0
4,2023-06-08T04:00,52.6,0.0
...,...,...,...
115,2023-06-12T19:00,65.2,1.2
116,2023-06-12T20:00,65.7,6.5
117,2023-06-12T21:00,65.9,4.2
118,2023-06-12T22:00,67.0,1.7


Add in some functions which show user requests being used for visualization


In [42]:
def get_forecast_daily(lat, long, forecast_d): #creating an api call for user's input
    params = {
        'latitude': lat,  
        'longitude': long, #have to convert city to lat,long
        'forecast_days': forecast_d, #number of days in forecast
        'temperature_unit': 'fahrenheit', # Change unit maybe later according to user pref
        'precipitation_unit': 'mm',
        'windspeed_unit': 'kmh', 
        'daily': 'temperature_2m_max',
        'timezone': 'America/New_York',  # Change according to your timezone - later user could maybe do this 
    }
    response = requests.get(url, params=params)
    return response.json()

1) User jane asks for 10 days NYC

In [43]:
# visual example
city = input("Enter a city ")  
latitude, longitude = get_cord(city)
if latitude and longitude:
    data = get_forecast_daily(latitude, longitude,10)
    forecast_df = pd.DataFrame(data['daily'])

In [46]:
#Create chart of temperature forecast for the next 10 days

import plotly.express as px
import plotly.graph_objects as go

forecast_df['time'] = pd.to_datetime(forecast_df['time']) #converting time column to a date/time data type
fig = px.line(forecast_df, x='time', y='temperature_2m_max') #defining x and y axis
fig.update_layout(
    title="Temperature Forecast",
    xaxis_title="Time",
    yaxis_title="Temperature (°F)",
)
fig.show()

2) Jane asks for rain and temp 

In [56]:
def get_forecast_daily(lat, long, forecast_d=10):
    params = {
        'latitude': lat,  
        'longitude': long,
        'forecast_days': forecast_d, 
        'temperature_unit': 'fahrenheit', 
        'precipitation_unit': 'mm',
        'windspeed_unit': 'kmh', 
        'daily': 'temperature_2m_min,temperature_2m_max,precipitation_probability_mean', #Added required parameters for precipitation
        'timezone': 'America/New_York',
    }
    response = requests.get(url, params=params)
    return response.json()
city = input("Enter a city ")  
latitude, longitude = get_cord(city)
if latitude and longitude:
    data = get_forecast_daily(latitude, longitude,10)
    forecast_df = pd.DataFrame(data['daily'])

#dividing min and max to create an average temperature column
forecast_df['avg_temperature'] = (forecast_df['temperature_2m_min'] + forecast_df['temperature_2m_max']) / 2 
forecast_df['time'] = pd.to_datetime(forecast_df['time'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast_df['time'], y=forecast_df['temperature_2m_max'], mode='lines', name='Max Temp'))
fig.add_trace(go.Scatter(x=forecast_df['time'], y=forecast_df['temperature_2m_min'], mode='lines', name='Min Temp'))
fig.add_trace(go.Scatter(x=forecast_df['time'], y=forecast_df['avg_temperature'], mode='lines', name='Avg Temp'))

high_precip = forecast_df[forecast_df['precipitation_probability_mean'] > 50] #defining what high precipitation is
low_precip = forecast_df[forecast_df['precipitation_probability_mean'] <= 50] #defining what low precipitation is

#adding precipitation to plot
fig.add_trace(go.Scatter(x=low_precip['time'], y=low_precip['precipitation_probability_mean'], mode='markers', name='Low Precipitation'))
fig.add_trace(go.Scatter(x=high_precip['time'], y=high_precip['precipitation_probability_mean'], mode='markers', marker=dict(color='red'), name='High Precipitation'))

fig.update_layout(
    title="Temperature Forecast",
    xaxis_title="Time",
    yaxis_title="Temperature (°F) / Precipitation Probability (%)",
    yaxis=dict(
        range=[0, 100],
    ),
)
fig.show()
